In [67]:
import itertools
import pandas as pd
import requests
from river import datasets
from river import utils
from river import evaluate
from river import forest
from river import metrics
from river import stream

url = "https://data.alpaca.markets/v1beta3/crypto/us/bars?symbols=BTC%2FUSD&timeframe=1Min&start=2023-08-01T00%3A00%3A00Z&limit=10000&sort=asc"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers).json()

data = response["bars"]['BTC/USD']

df = pd.DataFrame(data)

df["next_minute"] = df["c"].shift(-1)
df['target'] = (df["c"] < df["next_minute"]).astype(int)



x = df.drop(columns=['target','n','v','vw']).iloc[:-1000]
y = df['target'].iloc[:-1000]

feature_list = x.to_dict(orient='records')
target_list = y.tolist()

# # x_iter = x.to_L
# y_iter = y.items()

# x_slice = itertools.islice(x_iter, 1000)
# y_slice = itertools.islice(y_iter, 1000)

dataset = df.iloc[:-1000]

model = forest.AMFClassifier(
    n_estimators=10,
    use_aggregation=True,
    dirichlet=0.5,
    seed=1
)

metric = metrics.Accuracy()

evaluate.progressive_val_score(stream.iter_array(feature_list,target_list), model, metric)



Accuracy: 49.51%

In [50]:
dataset = datasets.Bananas().take(500)

dataset